In [1]:
# Import Libraries
import serial
import numpy as np
import time

In [2]:
import serial.tools.list_ports

In [ ]:
ports = serial.tools.list_ports.comports()

for port in ports:
    if port.serial_number == "85036313530351F0B141":
        arduino_port = port.device
    elif port.serial_number == "FT2OHLM4A":
        balance_port = port.device

In [ ]:
M = [None] * 8
A = [None] * 8
C = [None] * 8
X = [None] * 7
P = [None] * 1
m = [None] * 1

In [ ]:
def setupDispenser(dispenser,matrixComp,additiveComp):
    M[dispenser] = matrixComp
    A[dispenser] = additiveComp
    
def userInput(mass,matrixComp,A1Comp,A2Comp,A3Comp,A4Comp,A5Comp,A6Comp,A7Comp):
    m[0] = mass
    P[0] = matrixComp
    X[0] = A1Comp
    X[1] = A2Comp
    X[2] = A3Comp
    X[3] = A4Comp
    X[4] = A5Comp
    X[5] = A6Comp
    X[6] = A7Comp

In [ ]:
# Matrix - Additive (M-A) relation in each dispenser
setupDispenser(0,0.9,0.1)
setupDispenser(1,0.9,0.1)
setupDispenser(2,0.9,0.1)
setupDispenser(3,0.9,0.1)
setupDispenser(4,0.9,0.1)
setupDispenser(5,0.9,0.1)
setupDispenser(6,0.9,0.1)
setupDispenser(7,0.9,0.1)

In [ ]:
# USER INPUT: Desired proportion of PLA (P) and additives (X, Y, Z)
userInput(100,0.975,0,0.007,0.018,0,0,0,0)

In [ ]:
# Compute pellet proportions for desired output
# Σ Ci*(Mi*P + Ai*X)
for i in range(1,8):
    C[i] = X[i-1]/A[i]

In [ ]:
# Fill remaining with plastic
C[0] = P[0] - (C[1]*M[1] + C[2]*M[2] + C[3]*M[3] + C[4]*M[4] + C[5]*M[5] + C[6]*M[6] + C[7]*M[7])

In [ ]:
# Check user inputs
tol = 1e-3
if(abs(1- (P[0] + X[0] + X[1] + X[2] + X[3] + X[4] + X[5] + X[6])) < tol):
    if(C[0] >= 0):
        print('Concentrations add up.')
        print('**************************************')
        print('C1 =', C[0])
        print('C2 =', C[1])
        print('C3 =', C[2])
        print('C4 =', C[3])
    else:
        print('WARNING: Concentrations add up, but they are not feasible.')
        print('**************************************')
        print('Check user input before continuing.')
else:
    print('WARNING: Concentrations do not add up.')
    print('**************************************')
    print('Check user input before continuing.')

In [ ]:
concentrations = [C[0], C[1], C[2], C[3]]
minC = np.min([C for C in concentrations if C>tol])
#masked_concentrations = np.ma.masked_equal(concentrations, 0.0, copy=False)
#minC = masked_concentrations.min()
ratios = concentrations/minC 
print('The ratios for pellet dispensing will be: ', ratios)

In [ ]:
balance = serial.Serial(port=balance_port, baudrate=9600, bytesize=serial.SEVENBITS,
                       parity=serial.PARITY_EVEN, stopbits=serial.STOPBITS_ONE)

def tare(balance):
    # Set the balance measurement to 0.0 g
    try:
        balance.write(b'T\r\n')
    except:
        balance = serial.Serial(port=balance_port, baudrate=9600, bytesize=serial.SEVENBITS,
                       parity=serial.PARITY_EVEN, stopbits=serial.STOPBITS_ONE)
        balance.write(b'T\r\n')


def calib(balance):
    # Calibrate the balance automatically
    try:
        balance.write(b'C\r\n')
    except:
        balance = serial.Serial(port=balance_port, baudrate=9600, bytesize=serial.SEVENBITS,
                       parity=serial.PARITY_EVEN, stopbits=serial.STOPBITS_ONE)
        balance.write(b'C\r\n')

def measure(balance):
    try:
        trash = balance.read_all()
        balance.write(b'B\r\n')
        reading = balance.readline()
        weight = float(reading[:10])
    except:
        balance = serial.Serial(port=balance_port, baudrate=9600, bytesize=serial.SEVENBITS,
                       parity=serial.PARITY_EVEN, stopbits=serial.STOPBITS_ONE)
        trash = balance.read_all()
        balance.write(b'B\r\n')
        reading = balance.readline()
        weight = float(reading[:10])
    return weight

In [ ]:
arduino = serial.Serial(arduino_port, 9600)
time.sleep(2)

In [ ]:
arduino.write(b'8')

In [ ]:
measureAll = True
massAll = 0
mass = 0
masses = [[] for _ in range(8)]
i = 0
times = [[] for _ in range(8)]
start = time.time()

dispenser_switch = {0 : b'0', 1 : b'1', 2 : b'2', 3 : b'3', 
                    4 : b'4', 5 : b'5', 6 : b'6', 7 : b'7', 'stop' : b'8'}



while measureAll == True:
    for j in range(len(C)):
        if C[j] > tol:
            if masses[j][i] < ratios[j]*(i+1):
                measureNow = True
                tare(balance)
                time.sleep(2)
                arduino.write(dispenser_switch[j])
                while measureNow == True:
                    mass = measure(balance)
                    if mass + masses[j][i] >= ratios[j]*(i+1):
                        arduino.write(dispenser_switch['stop'])
                        time.sleep(4)
                        mass = measure(balance)
                        measureNow = False
                massAll = massAll + mass
                masses[j].append(masses[j][i]+mass)
                time.sleep(0.1)
            elif masses[j][i] >= ratios[j]*(i+1):
                arduino.write(dispenser_switch['stop'])
            times[j].append(time.time()-start)
            print("iteration ", i+1, "| t = ", times[j][i+1], "s | ", "mass : ", masses[j][i+1])                  
    if massAll > m[0]:
        measureAll = False
        arduino.write(dispenser_switch['stop'])
    i = i + 1

In [ ]:
arduino.write(dispenser_switch['stop'])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(finish1, mass1,label='PLA')
plt.plot(finish4,mass4,label='Boron Nitride')
plt.plot(finish3,mass3,label='Graphite')
plt.title("Progression of Sample")
plt.xlabel("Time [s]")
plt.ylabel("Mass [g]")
plt.legend()
plt.show()

In [ ]:
massAll

In [ ]:
error = X[1]abs((mass4[i] * 0.1)/(mass4[i] + mass1[i]))/(massAll)*100
accuracy = 100 - error
print('Accuracy:', '%.2f' % accuracy,'%')

rate = massAll/finish4[i]
rate = rate * 3600
print('Mass flow rate: ', '%.2f' % rate,'g/h')

In [3]:
import sys
sys.path.append('src')

In [4]:
from DispenserSetup import SetupDispenser

In [5]:
D, M, A = SetupDispenser("data/dispenser_setup.csv", "PLA")

Dispenser 4: 100.0% of PLA, 
Dispenser 5: 90.0% of PLA, 10.0% of GNP, 
Dispenser 6: 90.0% of PLA, 10.0% of BN, 


In [9]:
D

{'PLA': 4, 'GNP': 5, 'BN': 6}

In [10]:
M

array([0. , 0. , 0. , 1. , 0.9, 0.9, 0. , 0. ])

In [11]:
A

array([0. , 0. , 0. , 0. , 0.1, 0.1, 0. , 0. ])

In [12]:
import pandas as pd

In [13]:
pd.read_csv('./data/compositions_to_dispense.csv')

,Composition,PLA,GNP,CLO,BN
0,1,0.95,0.02,0.0,0.03


In [14]:
M*1+

array([0.  , 0.  , 0.  , 0.  , 0.09, 0.09, 0.  , 0.  ])